In [ ]:
# === 1. KÖRNYEZET BEÁLLÍTÁSA ===
# Könyvtárak telepítése és importálása
%pip install -U torch sentence-transformers accelerate pyarrow pandas tqdm transformers runpod python-dotenv

import pandas as pd
import numpy as np
import gc
import json
import pyarrow.parquet as pq
import torch
import time
from tqdm.auto import tqdm
from typing import List
from pathlib import Path
import os
import runpod
from dotenv import load_dotenv

# .env fájl betöltése a RUNPOD_API_KEY betöltéséhez
load_dotenv()

# GPU-specifikus optimalizációk már nem szükségesek, mivel a számítás a felhőben történik
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# torch.backends.cudnn.benchmark = True
# torch.backends.cuda.matmul.allow_tf32 = True

print(f"CUDA elérhető lokálisan: {torch.cuda.is_available()}")

In [ ]:
# === 2. KONFIGURÁCIÓ ===
# RunPod Serverless API-hoz igazított konfiguráció.

from pathlib import Path
import os

# API kulcs és Endpoint ID beolvasása
RUNPOD_API_KEY = os.getenv("RUNPOD_API_KEY")
RUNPOD_ENDPOINT_ID = "5hjxb1eht972gw" # A RunPod UI-ból kimásolt endpoint ID

if not RUNPOD_API_KEY:
    raise ValueError("A RUNPOD_API_KEY környezeti változó nincs beállítva! Állítsd be egy .env fájlban.")

# Bemeneti és kimeneti fájlok
INPUT_CSV_PATH = Path("../processed_data/cleaned_data_for_embedding.csv")
OUTPUT_PARQUET_PATH = Path("../processed_data/documents_with_embeddings_api.parquet")

# Embedding és batch méret beállítások
EMBEDDING_DIMENSION = 1024
# Kliens oldali batch méret, egy kérésben ennyi szöveg megy el az API-nak
BATCH_SIZE = 256

print(f"RunPod Endpoint ID: {RUNPOD_ENDPOINT_ID}")
print(f"Input: {INPUT_CSV_PATH}")
print(f"Output: {OUTPUT_PARQUET_PATH}")
print(f"Batch méret: {BATCH_SIZE}")

In [ ]:
# === 3. EMBEDDING GENERÁTOR OSZTÁLY (RUNPOD API) ===
class EmbeddingGeneratorAPI:
    def __init__(self, api_key: str, endpoint_id: str, batch_size: int, dimension: int):
        self.api_key = api_key
        self.endpoint_id = endpoint_id
        self.batch_size = batch_size
        self.dimension = dimension
        runpod.api_key = self.api_key
        self.endpoint = runpod.Endpoint(self.endpoint_id)
        print(f"RunPod API generátor inicializálva a(z) '{self.endpoint_id}' endpoint-ra.")

    def load_model(self):
        # Nincs szükség modell betöltésére, az endpoint kezeli.
        # Ez a metódus a kompatibilitás miatt marad.
        print("A modell a RunPod szerveren fut, nincs szükség lokális betöltésre.")

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        all_embeddings = []
        
        # Feldolgozás batch-ekben
        for i in tqdm(range(0, len(texts), self.batch_size), desc="Embedding generálás (API)"):
            batch_texts = texts[i:i + self.batch_size]
            
            request = {
                "input": {
                    "texts": batch_texts,
                    "normalize_embeddings": True
                }
            }
            
            try:
                # Szinkron kérés küldése, ami megvárja a választ
                result = self.endpoint.run_sync(request, timeout=600) # 10 perc timeout
                batch_embeddings = result['output']['embeddings']
                all_embeddings.extend(batch_embeddings)
            except Exception as e:
                print(f"Hiba a RunPod API hívás során a(z) {i}-edik elemnél: {e}")
                # Hibakezelés: üres embeddingekkel töltjük fel a hibás batch helyét
                error_placeholder = np.zeros((len(batch_texts), self.dimension), dtype=np.float32)
                all_embeddings.extend(error_placeholder.tolist())
                continue
        
        embeddings_array = np.array(all_embeddings, dtype=np.float32)

        # Biztonsági ellenőrzés a dimenzióra
        if embeddings_array.shape[1] != self.dimension:
             print(f"Figyelmeztetés: Váratlan embedding dimenzió: {embeddings_array.shape[1]}. Korrekció {self.dimension}-ra.")
             # Itt vagy hibát dobunk, vagy megpróbáljuk korrigálni, ha lehetséges
             # Most feltételezzük, hogy ez egy kritikus hiba
             raise ValueError(f"Váratlan embedding dimenzió: {embeddings_array.shape[1]}")
            
        return embeddings_array

    def _cleanup_memory(self):
        # Nincs szükség GPU memória takarítására
        pass

In [ ]:
# === 4. FŐ FELDOLGOZÁSI FOLYAMAT ===
def create_metadata_json(row: pd.Series) -> str:
    """Létrehoz egy JSON stringet a sor metaadataiból."""
    metadata_cols = [col for col in row.index if col not in ['text', 'embedding']]
    metadata_dict = row[metadata_cols].dropna().to_dict()
    return json.dumps({k: str(v) for k, v in metadata_dict.items()}, ensure_ascii=False)

def main():
    print("Feldolgozás indítása a RunPod API-val...")
    start_time = time.time()

    # Bemeneti adatok beolvasása
    if not INPUT_CSV_PATH.exists():
        raise FileNotFoundError(f"Hiba: A bemeneti fájl nem található: {INPUT_CSV_PATH}")
    
    print(f"Bemeneti CSV beolvasása: {INPUT_CSV_PATH}")
    # Egyszerűsített beolvasás, a C motor használatával
    df = pd.read_csv(INPUT_CSV_PATH)
    print(f"{len(df):,} sor sikeresen beolvasva.")

    # Szövegek kinyerése
    df['text'] = df['text'].fillna('')
    texts_to_process = df['text'].astype(str).tolist()
    
    if not texts_to_process:
        print("Nincs feldolgozandó szöveg a bemeneti fájlban.")
        return

    # Embedding generátor inicializálása és "modell betöltése"
    generator = EmbeddingGeneratorAPI(RUNPOD_API_KEY, RUNPOD_ENDPOINT_ID, BATCH_SIZE, EMBEDDING_DIMENSION)
    generator.load_model() # Kompatibilitási hívás, valójában nem csinál semmit

    # Embedding generálás a teljes adathalmazon
    print("Embedding generálás megkezdése a RunPod API-n keresztül...")
    embeddings = generator.generate_embeddings(texts_to_process)
    
    # Memória takarítás a nagy művelet után
    generator._cleanup_memory()

    # Eredmények hozzáadása a DataFrame-hez
    if len(embeddings) == len(df):
        df['embedding'] = list(embeddings)
    else:
        print(f"KRITIKUS HIBA: Az embeddingek száma ({len(embeddings)}) nem egyezik a DataFrame sorainak számával ({len(df)}). A program leáll.")
        return

    # Metaadatok generálása
    print("Metaadat JSON generálása...")
    df['metadata_json'] = [create_metadata_json(row) for _, row in tqdm(df.iterrows(), total=len(df), desc="Metaadat JSON")]

    # Kimeneti DataFrame és mentés Parquet formátumba
    final_df = df[['doc_id', 'text', 'embedding', 'metadata_json']]
    OUTPUT_PARQUET_PATH.parent.mkdir(parents=True, exist_ok=True)
    print(f"Eredmények mentése a Parquet fájlba: {OUTPUT_PARQUET_PATH}")
    final_df.to_parquet(OUTPUT_PARQUET_PATH, index=False, compression='snappy')
    
    # Összegzés
    total_rows_processed = len(final_df)
    total_time_seconds = time.time() - start_time
    rows_per_second = total_rows_processed / total_time_seconds if total_time_seconds > 0 else 0
    
    print("\n" + "="*50)
    print("✅ FELDOLGOZÁS BEFEJEZVE")
    print(f"📄 Kimeneti fájl: {OUTPUT_PARQUET_PATH}")
    print(f"⏱️ Teljes idő: {total_time_seconds:.2f} másodperc ({total_time_seconds / 60:.2f} perc)")
    print(f"📊 Feldolgozott sorok: {total_rows_processed:,}")
    print(f"⚡ Átlagos sebesség: {rows_per_second:.2f} sor/mp")
    print("="*50)

# Fő folyamat futtatása
main()

In [ ]:
# === 5. VALIDÁCIÓ ===
print("Kimeneti Parquet fájl validálása...")

if OUTPUT_PARQUET_PATH.exists():
    try:
        parquet_file = pq.ParquetFile(OUTPUT_PARQUET_PATH)
        file_num_rows = parquet_file.metadata.num_rows
        file_size_mb = OUTPUT_PARQUET_PATH.stat().st_size / (1024 * 1024)
        
        df_sample = pd.read_parquet(OUTPUT_PARQUET_PATH, engine='pyarrow').head(5)
        sample_embedding = df_sample['embedding'].iloc[0]
        
        print("\n✅ VALIDÁCIÓ SIKERES!")
        print(f"  Fájl méret: {file_size_mb:.2f} MB")
        print(f"  Sorok száma: {file_num_rows:,}")
        print(f"  Oszlopok: {df_sample.columns.tolist()}")
        print(f"  Első embedding dimenziója: {len(sample_embedding)}")
        print("\n--- Minta Adatsor ---")
        display(df_sample)
        
    except Exception as e:
        print(f"Hiba a Parquet fájl validálása közben: {e}")
        print(f"\n❌ HIBA a validáció során: {e}")
else:
    print("A kimeneti Parquet fájl nem jött létre.")
    print("\n❌ HIBA: A kimeneti fájl nem található!")